In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [10]:
# Read the data
df = pd.read_csv('../data/permis-construction.csv')


/var/folders/8j/ybqbv42143jfglvsf83t5s_c0000gn/T/ipykernel_27344/2165301696.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/permis-construction.csv')


In [11]:
# print the column names
print(list(df.columns))

['no_demande', 'id_permis', 'date_debut', 'date_emission', 'emplacement', 'arrondissement', 'code_type_base_demande', 'description_type_demande', 'description_type_batiment', 'description_categorie_batiment', 'nature_travaux', 'nb_logements', 'cout_traveaux_estimes', 'cout_permis_emis', 'LETTRE_DEBUT', 'LETTRE_FIN', 'longitude', 'latitude', 'loc_x', 'loc_y']


In [12]:
# rename the columns from french to english
new_columns = ['request_number', 'license_id', 'start_date', 'issue_date', 'location', 'district', 'request_base_code', 'request_type_description', 'building_type_description', 'building_category_description', 'work_nature', 'number_of_units', 'estimated_work_cost', 'issued_permit_cost', 'LETTER_START', 'LETTER_END', 'longitude', 'latitude', 'loc_x', 'loc_y']
df.columns = new_columns

In [14]:
# drop unnecessary columns
columns_to_drop = ['request_number','license_id', 'location', 'district', 'request_type_description', 'building_category_description', 'work_nature', 'LETTER_START','LETTER_END']
new_df = df
new_df.drop(columns_to_drop, axis=1, inplace=True)

In [15]:
new_df.head(2)


,start_date,issue_date,request_base_code,building_type_description,number_of_units,estimated_work_cost,issued_permit_cost,longitude,latitude,loc_x,loc_y
0,1994-05-16,1994-05-18,CO,Résidentiel,1.0,110000.0,380.0,-73.869902,45.505224,275894.73,5040655.81
1,1997-08-06,1997-08-07,TR,Résidentiel,0.0,8500.0,34.0,-73.873945,45.501934,275577.10,5040291.58


## Creating Data Column for Permit Approval Time


In [16]:
# convert issue date to datetime
data_df = new_df
data_df['issue_date'] = pd.to_datetime(data_df['issue_date'])
data_df['start_date'] = pd.to_datetime(data_df['start_date'])


In [17]:
# get approval time in days from start date and issue date
data_df['approval_time'] = data_df['issue_date'] - data_df['start_date']


In [18]:
data_df.head(5)


,start_date,issue_date,request_base_code,building_type_description,number_of_units,estimated_work_cost,issued_permit_cost,longitude,latitude,loc_x,loc_y,approval_time
0,1994-05-16,1994-05-18,CO,Résidentiel,1.0,110000.0,380.0,-73.869902,45.505224,275894.73,5040655.81,2 days
1,1997-08-06,1997-08-07,TR,Résidentiel,0.0,8500.0,34.0,-73.873945,45.501934,275577.10,5040291.58,1 days
2,1997-08-08,1997-09-24,DE,Résidentiel,-1.0,1000.0,40.0,-73.930681,45.492434,271137.69,5039258.11,47 days
3,1997-08-08,1997-09-24,DE,Commercial,0.0,1000.0,40.0,-73.930681,45.492434,271137.69,5039258.11,47 days
4,1997-08-08,1997-09-24,CO,Résidentiel,1.0,150000.0,450.0,-73.930681,45.492434,271137.69,5039258.11,47 days


In [19]:
approval_time_df = data_df
approval_time_df.drop(['start_date', 'issue_date','loc_x', 'loc_y'], axis=1, inplace=True)

In [20]:
approval_time_df.drop(['estimated_work_cost', 'issued_permit_cost'], axis=1, inplace=True)

In [21]:
approval_time_df.head(5)

,request_base_code,building_type_description,number_of_units,longitude,latitude,approval_time
0,CO,Résidentiel,1.0,-73.869902,45.505224,2 days
1,TR,Résidentiel,0.0,-73.873945,45.501934,1 days
2,DE,Résidentiel,-1.0,-73.930681,45.492434,47 days
3,DE,Commercial,0.0,-73.930681,45.492434,47 days
4,CO,Résidentiel,1.0,-73.930681,45.492434,47 days


In [22]:
# drop rows with negative values in the 'number_of_units' column
approval_time_df = approval_time_df[approval_time_df.number_of_units >= 0].reset_index(drop=True)
approval_time_df.head(5)



,request_base_code,building_type_description,number_of_units,longitude,latitude,approval_time
0,CO,Résidentiel,1.0,-73.869902,45.505224,2 days
1,TR,Résidentiel,0.0,-73.873945,45.501934,1 days
2,DE,Commercial,0.0,-73.930681,45.492434,47 days
3,CO,Résidentiel,1.0,-73.930681,45.492434,47 days
4,CO,Résidentiel,0.0,-73.930681,45.492434,47 days


In [23]:
# remove rows with missing values
approval_time_df.dropna(inplace=True)
approval_time_df.isnull().sum()

request_base_code            0
building_type_description    0
number_of_units              0
longitude                    0
latitude                     0
approval_time                0
dtype: int64

In [24]:
approval_time_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479126 entries, 0 to 495152
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype          
---  ------                     --------------   -----          
 0   request_base_code          479126 non-null  object         
 1   building_type_description  479126 non-null  object         
 2   number_of_units            479126 non-null  float64        
 3   longitude                  479126 non-null  float64        
 4   latitude                   479126 non-null  float64        
 5   approval_time              479126 non-null  timedelta64[ns]
dtypes: float64(3), object(2), timedelta64[ns](1)
memory usage: 25.6+ MB


In [25]:
# check for duplicated rows
print(f'Duplicated rows: {approval_time_df.duplicated().sum()} out of {len(approval_time_df)} rows')
# drop duplicated rows
approval_time_df.drop_duplicates(inplace=True)

Duplicated rows: 53360 out of 479126 rows


In [26]:
print(f'Duplicated rows: {approval_time_df.duplicated().sum()} out of {len(approval_time_df)} rows')

Duplicated rows: 0 out of 425766 rows


In [27]:
# remove the days from the 'days_to_approval' column and convert to integer
approval_time_df['approval_time'] = approval_time_df['approval_time'].dt.days.astype('int')


In [28]:
approval_time_df.head(5)

,request_base_code,building_type_description,number_of_units,longitude,latitude,approval_time
0,CO,Résidentiel,1.0,-73.869902,45.505224,2
1,TR,Résidentiel,0.0,-73.873945,45.501934,1
2,DE,Commercial,0.0,-73.930681,45.492434,47
3,CO,Résidentiel,1.0,-73.930681,45.492434,47
4,CO,Résidentiel,0.0,-73.930681,45.492434,47


## Convert the data into training and test set and store the test set separately in order to check our model.

In [29]:

# absolute value of 80% of the data will be used for training
training_length = np.abs(0.8*len(approval_time_df))
training_df = approval_time_df.head(int(training_length))
len(training_df)
# remaining 20% of the data will be used for testing
testing_df = approval_time_df.tail(len(approval_time_df) - len(training_df))
len(testing_df)

85154

In [30]:
# save the testing csv file
testing_df.to_csv('../data/testing.csv', index=False)

In [31]:
# encode building type description
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Create a OneHotEncoder object
# onehot_encoder = OneHotEncoder()
# # Encode the building types
# encoded_building_types = onehot_encoder.fit_transform(training_df['building_type_description'].values.reshape(-1,1)).toarray()
# print(encoded_building_types)

# Create a LabelEncoder object
label_encoder = LabelEncoder()
# Fit and transform the building types
encoded_building_types = label_encoder.fit_transform(training_df['building_type_description'])
print(encoded_building_types)

[24 24  4 ... 24 13 24]


In [32]:
# Create a dataframe from the encoded building types
building_types_df = pd.DataFrame(encoded_building_types, columns=['building_type'])
# Concatenate the dataframes
training_df = pd.concat([training_df, building_types_df], axis=1)
training_df.head(5)

,request_base_code,building_type_description,number_of_units,longitude,latitude,approval_time,building_type
0,CO,Résidentiel,1.0,-73.869902,45.505224,2.0,24.0
1,TR,Résidentiel,0.0,-73.873945,45.501934,1.0,24.0
2,DE,Commercial,0.0,-73.930681,45.492434,47.0,4.0
3,CO,Résidentiel,1.0,-73.930681,45.492434,47.0,24.0
4,CO,Résidentiel,0.0,-73.930681,45.492434,47.0,24.0


In [33]:
# # add the encoded building types to the dataframe
# building_types_df = pd.DataFrame(encoded_building_types, columns=['building_type_'+str(int(i)) for i in range(encoded_building_types.shape[1])])
# training_df = pd.concat([training_df, building_types_df], axis=1)

# drop the building type description column
training_df.drop('building_type_description', axis=1, inplace=True)
training_df.head(5)

,request_base_code,number_of_units,longitude,latitude,approval_time,building_type
0,CO,1.0,-73.869902,45.505224,2.0,24.0
1,TR,0.0,-73.873945,45.501934,1.0,24.0
2,DE,0.0,-73.930681,45.492434,47.0,4.0
3,CO,1.0,-73.930681,45.492434,47.0,24.0
4,CO,0.0,-73.930681,45.492434,47.0,24.0


In [34]:
training_df['request_base_code'].unique()

array(['CO', 'TR', 'DE', 'CA', nan], dtype=object)

In [35]:
#  encode request base code
# Create a LabelEncoder object
label_encoder = LabelEncoder()
# Fit and transform the request base code
encoded_request_base_code = label_encoder.fit_transform(training_df['request_base_code'])
print(encoded_request_base_code)


[1 3 2 ... 4 4 4]


In [36]:
# Create a dataframe from the encoded request base code
request_base_code_df = pd.DataFrame(encoded_request_base_code, columns=['request_code'])
# Concatenate the dataframes
training_df = pd.concat([training_df, request_base_code_df], axis=1)

# drop the request base code column
training_df.drop('request_base_code', axis=1, inplace=True)
training_df.head(5)


,number_of_units,longitude,latitude,approval_time,building_type,request_code
0,1.0,-73.869902,45.505224,2.0,24.0,1.0
1,0.0,-73.873945,45.501934,1.0,24.0,3.0
2,0.0,-73.930681,45.492434,47.0,4.0,2.0
3,1.0,-73.930681,45.492434,47.0,24.0,1.0
4,0.0,-73.930681,45.492434,47.0,24.0,1.0


In [28]:
# # onhot encode the request base code
# encoded_request_base_code = onehot_encoder.fit_transform(training_df['request_base_code'].values.reshape(-1,1)).toarray()
# # add the encoded request base code to the dataframe
# request_base_code_df = pd.DataFrame(encoded_request_base_code, columns=['request_base_code_'+str(int(i)) for i in range(encoded_request_base_code.shape[1])])
# training_df = pd.concat([training_df, request_base_code_df], axis=1)
# # drop the request base code column
# training_df.drop('request_base_code', axis=1, inplace=True)
# training_df.head(5)

In [37]:
# put approval_time column at the end of the dataframe
cols = list(training_df.columns.values)
cols.pop(cols.index('approval_time'))
training_df = training_df[cols+['approval_time']]
training_df.head(5)

,number_of_units,longitude,latitude,building_type,request_code,approval_time
0,1.0,-73.869902,45.505224,24.0,1.0,2.0
1,0.0,-73.873945,45.501934,24.0,3.0,1.0
2,0.0,-73.930681,45.492434,4.0,2.0,47.0
3,1.0,-73.930681,45.492434,24.0,1.0,47.0
4,0.0,-73.930681,45.492434,24.0,1.0,47.0


In [38]:
# save the dataframe to csv
training_df.to_csv('../data/permit-time-training.csv', index=False)

In [40]:
# read the dataframe from csv
df = pd.read_csv('../data/permit-time-training.csv')
df.head(5)

,number_of_units,longitude,latitude,building_type,request_code,approval_time
0,1.0,-73.869902,45.505224,24.0,1.0,2.0
1,0.0,-73.873945,45.501934,24.0,3.0,1.0
2,0.0,-73.930681,45.492434,4.0,2.0,47.0
3,1.0,-73.930681,45.492434,24.0,1.0,47.0
4,0.0,-73.930681,45.492434,24.0,1.0,47.0
